In [ ]:
import re
str1 = '[{timestamp}] {}txAtt:{txAtt:d}, {}torTemperature:{torTemperature:d} {}avgIMpa0:{avgIMpa0:d} {}'
re.findall('\{(.*?)\}', str1)

In [ ]:
import json
import uuid
branch = ['A','B','C','D','E','F','G','H','I','J','K','L']
config = {'search':[], 'insight':[], 'chart':[], 'statistic':[]}
for index in range(12):
    item = {}
    item['role_path'] = 'root.' + 'branch' + str(branch[index]) + '.search_branch' + str(branch[index])
    item['identifier'] = 'search_branch' + str(branch[index])
    item['desc'] = 'Branch ' + str(branch[index]) + ' common use KeyValue and Mark'
    item['exp_search'] = '(txlProcBranch'+str(branch[index])+'|TxBranchCtrl'+str(branch[index])+').*(txAtt|Pma|linearization fault)'
    item['exp_extract'] = [
                        "{}[{timestamp}] {}txAtt:{txAtt:d}, {}torTemperature:{torTemperature:d} {}avgIMpa0:{avgIMpa0:d} {}",
                        # "{}[{timestamp}] {}Pma:{Pma:f}[{}DpdPma:{DpdPma:f}[{}Pmb:{Pmb:f}[{}TorPmb:{TorPmb:f}{}avgTxPma:{avgTxPma:f} {}",
                        "{}[{timestamp}] {}"
                        ]
    item['exp_mark'] = [{"abbr":"LF","exp":"linearization fault","color":"#f00000"}]
    item['is_active'] = False
    item['is_case_sensitive'] = True
    item['forward_rows'] = 0
    item['backward_rows'] = 0
    if index == 0:
        item['is_active'] = True
    config['search'].append(item)

keys = ['txAtt', 'torTemperature', 'avgIMpa0', 'LF']
for search_config in config['search']:
    item = {}
    item['role_path'] = search_config['role_path'].replace('search', 'chart')
    item['identifier'] = search_config['identifier'].replace('search', 'chart')
    item['desc'] = item['identifier'] + ' all KeyValue and Mark'
    item['check'] = False
    item['key_value_tree'] = {'namespace': 'KeyValue', 'name':'Key Value', 'check':False, 'children':[]}
    for search_config2 in config['search']:
        search_atom = {}
        search_atom['namespace'] = search_config2['identifier']
        search_atom['name'] = search_config2['identifier']
        search_atom['check'] = False
        search_atom['children'] = []
        for key in keys:
            if search_config['identifier'] == search_config2['identifier']:
                search_atom['children'].append({'name': key, 'check': True})
            else:
                search_atom['children'].append({'name': key, 'check': False})
        item['key_value_tree']['children'].append(search_atom)
    config['chart'].append(item)

# for index in range(12):
#     item = {}
#     item['name'] = 'insight_branch' + str(branch[index])
#     item['desc'] = 'Branch Insight' + str(branch[index])
#     item['exp_search'] = '(txlProcBranch'+str(branch[index])+'|TxBranchCtrl'+str(branch[index])+')'
#     item['exp_extract'] = "{}[{timestamp}]{}, msg = {msg}"
#     item['exp_mark'] = {"name":"LF","exp":"linearization fault","color":"#f00000"}
#     item['is_case_sensitive'] = True
#     item['forward_rows'] = 0
#     item['backward_rows'] = 0
#     config['insight'].append(item)

# for index in range(12):
#     item = {}
#     item['name'] = 'statistic_branch' + str(branch[index])
#     item['desc'] = 'Branch ' + str(branch[index]) + ' statistic txAtt'
#     item['code'] = "self.result = self.text_file_model.data['" + 'search_branch' + str(branch[index]) + "'].res_key_value['txAtt']['global_index']"
#     config['statistic'].append(item)
    
json_object = json.dumps(config)
with open("E:\\projects\\ericsson_flow\\new_files\\config.ecfg", "w") as outfile:
    outfile.write(json_object)

In [ ]:
import json
import uuid
import re

def add_search_config(parent, express, processes, keyword, mark):
    config = []
    
    item = {}
    item['identifier'] = f'{parent}_ErrorFaultABN'
    identifier = item['identifier']
    item['role_path'] = f'root.{parent}.EFA.{identifier}'

    item['desc'] = parent + ' Error Fault ABN'
    item['exp_search'] = parent + '.*(error| fault|ABN)'
    item['exp_extract'] = [
                            parent + express
                        ]
    item['exp_mark'] = [
                            {"abbr":"ER","exp":"error","color":"#f00000"},
                            {"abbr":"FA","exp":" fault","color":"#f00000"},
                            {"abbr":"ABN","exp":"ABN","color":"#ffff00"}
                        ]
    item['is_active'] = True
    item['is_case_sensitive'] = False
    item['forward_rows'] = 0
    item['backward_rows'] = 0
    config.append(item)

    for index, process in enumerate(processes):
        item = {}
        name = process.replace(',','_')
        item['identifier'] = f'{parent}_{name}'
        identifier = item['identifier']
        item['role_path'] = f'root.{parent}.{identifier}' if (keyword not in process) else f'root.{parent}.{process}.{identifier}'
        item['desc'] =  parent + ' ' + process + ' common use KeyValue and Mark'
        item['exp_search'] = f'{parent}.*{process}'
        item['exp_extract'] = [
                                parent + express
                            ]
        item['exp_mark'] = mark if (keyword in process) else []
        item['is_active'] = False
        item['is_case_sensitive'] = True
        item['forward_rows'] = 0
        item['backward_rows'] = 0
        config.append(item)
    return config
        
def add_chart_config(tmp_search):
    config = []
    for search_config in tmp_search:

        if len(search_config['exp_mark']) == 0:
            continue

        item = {}
        item['role_path'] = search_config['role_path'] + '_Chart'
        item['identifier'] = search_config['identifier'] + '_Chart'
        item['desc'] = item['identifier'] + ' all KeyValue and Mark'
        item['check'] = False
        item['key_value_tree'] = {'namespace': 'KeyValue', 'name':'Key Value', 'check':False, 'children':[]}
        for search_config2 in tmp_search:
            search_atom = {}
            search_atom['namespace'] = search_config2['identifier']
            search_atom['name'] = search_config2['identifier']
            search_atom['check'] = False
            search_atom['children'] = []
            if len(search_config2['exp_mark']) > 0:
                keys = []
                for mark in search_config2['exp_mark']:
                    keys.append(mark['abbr'])
                    
                for key in keys:
                    if search_config['identifier'] == search_config2['identifier']:
                        search_atom['children'].append({'name': key, 'check': True})
                    else:
                        search_atom['children'].append({'name': key, 'check': False})
                        
            item['key_value_tree']['children'].append(search_atom)
        config.append(item)
    return config

config = {'search':[], 'insight':[], 'chart':[], 'statistic':[]}

processes = []
with open("D:\\projects\\ericsson_flow\\ErrorLog\\DOT TE LOG.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        res = re.findall(' \((.*?)\): ', line)
        if len(res) > 0:
            processes.append(res[0]) 
processes = list(set(processes))
processes.sort()
config['search'].extend(add_search_config('DOT_TE_LOG', ":{timestamp} ({}", processes, 'kkkkkkkkkkkkk', []))

processes = []
with open("D:\\projects\\ericsson_flow\\ErrorLog\\DU1 TE LOG.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        res = re.findall('procname = \"(.*?)\"', line)
        if len(res) > 0:
            processes.append(res[0])
processes = list(set(processes))
processes.sort()
config['search'].extend(add_search_config('DU1_TE_LOG', ":[{timestamp}]{}", processes, 'kkkkkkkkkkkkk', []))
config['search'].extend(add_search_config('DU2_TE_LOG', ":[{timestamp}]{}", processes, 'kkkkkkkkkkkkk', []))

processes = []
with open("D:\\projects\\ericsson_flow\\ErrorLog\\RU1 TE LOG.txt", "r") as f:
    lines = f.readlines()
    for line in lines:
        res = re.findall(', { \"(.*?)\"', line)
        if len(res) > 0:
            processes.append(res[0])
processes = list(set(processes))
processes.sort()
mark = [
            {"abbr":"ED","exp":"EVENT_DEACTIVATE","color":"#759aa0"},
            {"abbr":"ER","exp":"EVENT_RELEASE","color":"#e69d87"},
            {"abbr":"ES","exp":"EVENT_SETUP","color":"#8dc1a9"},
            {"abbr":"EA","exp":"EVENT_ACTIVATE","color":"#ea7e53"},
            {"abbr":"IND","exp":"IND_ACTIVATE","color":"#eedd78"}
        ]
tmp_search = add_search_config('RU1_TE_LOG', '{}[{timestamp}]{}', processes, 'TxBranchCtrl', mark)
config['search'].extend(tmp_search)
config['chart'].extend(add_chart_config(tmp_search))

tmp_search = add_search_config('RU2_TE_LOG', '{}[{timestamp}]{}', processes, 'TxBranchCtrl', mark)
config['search'].extend(tmp_search)
config['chart'].extend(add_chart_config(tmp_search))

json_object = json.dumps(config)
with open("D:\\projects\\ericsson_flow\\new_files\\DotConfigFull.ecfg", "w") as outfile:
    outfile.write(json_object)

In [ ]:
import re
from parse import parse

str1 = 'RU2_TE_LOG:BXP_2: [2023-02-23 10:31:25.766459124] com_ericsson_trithread:INFO: { 0 }, { "TxBranchCtrlC", "logManager.cc:37", "Branch does not support different power level (txChangeCycleStatePrepare.cc:517)" }'
# str2 = 'BXP_3: [2022-12-10 15:55:26.739019220] (+0.000025340) radio6626 com_ericsson_trithread:INFO: { cpu_id = 1 }, { process = "TxBranchCtrlB", fileAndLine = "txChangeCycleHelper.cc:264", msg = "Txl branch J restart due to txL linearization fault!" }'
str1 = "	fru_2051: [230201 120650]  144: PSU enters: DC_AISG_U:0 exceeds the limists. Value(mV): 587, limits[EXCEPTIONAL_HI,NORMAL_HI, NORMAL_LOW, EXCEPTIONAL_LOW](13125,13125,11875,11875)"
# exp = "\[%{TIMESTAMP:time}\] \(%{STRING:cost}\) "
# exp = "\[%{TIMESTAMP:time}\] %{DROP:tmp}Pma:%{FLOAT:Pma0}\[%{DROP:tmp1}DpdPma:%{FLOAT:DpdPma0}\[%{DROP:tmp2}Pmb:%{FLOAT:Pmb}, TorPmb:%{FLOAT:TorPmb0}\[%{FLOAT:TorPmb1} %{FLOAT:TorPmb2}\] "
exp_extract = '{}[{timestamp}]{}PSU enters{}'
r_extract = parse(exp_extract, str1)
print(r_extract)

In [ ]:
import re

# processes = []
# with open("D:\\projects\\ericsson_flow\\new_files\\4449\\faillog 4449 ULSA.txt", "r") as f:
#     lines = f.readlines()
#     for line in lines:
#         res = re.findall("{ \"(.*?)\",", line)
#         if len(res) > 0:
#             # if ('BranchCtrl' not in res[0]) & ('PhaseCtrl' not in res[0]):
#             processes.append(res[0])
            
# with open("D:\\projects\\ericsson_flow\\new_files\\4449\\passlog 4449 ULSA.txt", "r") as f:
#     lines = f.readlines()
#     for line in lines:
#         res = re.findall("{ \"(.*?)\",", line)
#         if len(res) > 0:
#             # if ('BranchCtrl' not in res[0]) & ('PhaseCtrl' not in res[0]):
#             processes.append(res[0])
            
# processes = list(set(processes))
# processes.sort()
# processes
processes = [
 'AntennaModule',
 'EquipCtrl',
 'PaSrvDC',
 'PimcTdiHandler',
 'RICRAI_SERVER',
 'RxBranchCtrlA',
 'RxBranchCtrlB',
 'RxBranchCtrlC',
 'RxBranchCtrlD',
 'RxRmcService',
 'TorLoSrvCD',
 'TxBranchCtrlC',
 'TxBranchCtrlD',
 'TxCoordinationSrvCD',
 'TxLoSrvC',
 'TxLoSrvD',
 'TxTimingPhaseCtrlC',
 'antpServerProc_',
 'bcProc',
 'brhProc',
 'carrierListElogScheduler.cc',
 'carrierResourceHandlerImpl.cc',
 'child.c',
 'cmd_proc',
 'cmd_proc(COLI)',
 'cpriCtrlProc',
 'cprilhmd.c',
 'crhCarrierCtrl.cc',
 'crhCarrierListHandler.cc',
 'crhProxyProc',
 'ecp_x11.c',
 'faultManagerProc',
 'iqIqcSwitchingAlgorithmCpri.cc',
 'iqIqcSwitchingResourceHandlerIf.cc',
 'iqIqcSwitchingResourceHandlerMs.cc',
 'ledProc',
 'libecp_x11.c',
 'lrciRasCapDbFacadeAbfaas.cc',
 'lrciRasCapDbFacadeBase.cc',
 'lrciRasCapabilities.cc',
 'radioTrDcServer',
 'rfPort.cc',
 'rftProc',
 'rhd-lh.c',
 'rmcProc',
 'rtsCtrl',
 'rtsCtrl_requestProcessor',
 'rxBranchTimingInfoReporter.cc',
 'stimenotifyd.c',
 'tmoSchedulerEqp',
 'tmoSchedulerTx',
 'trDcProc',
 'txBranchReporterCrh.cc',
 'txlProcBranchC',
 'txlProcBranchD',
 'ulh_hdlc.c']

In [ ]:
import json
import uuid

product_name = 'Radio4449'
processes = processes
config = {'search':[], 'insight':[], 'chart':[], 'statistic':[]}

item = {}
item['role_path'] = f'{product_name}.TimedoutErrorFaultABN'
item['identifier'] = 'TimedoutErrorFaultABN'
item['desc'] = 'Error Fault ABN Timedout'
item['exp_search'] = 'error| fault|ABN|Timed out'
item['exp_extract'] = [
                    "[{timestamp}] {}" 
                    ]
item['exp_mark'] = [
                        {"abbr":"TO","exp":"Timed out","color":"#FFB6C1"},
                        {"abbr":"ER","exp":"error","color":"#f00000"},
                        {"abbr":"LF","exp":"linearization fault","color":"#f00000"},
                        {"abbr":"ABN","exp":"ABN:","color":"#ffff00"}
                    ]
item['is_active'] = True
item['is_case_sensitive'] = True
item['forward_rows'] = 0
item['backward_rows'] = 0
config['search'].append(item)

for index, process in enumerate(processes):
    item = {}
    item['role_path'] = f'{product_name}.' + process.replace('.','_').replace('(','_').replace(')','_')
    item['identifier'] = process
    item['desc'] =  process + ' common use KeyValue and Mark'
    item['exp_search'] = '"'+process.replace('(','\\(').replace(')','\\)')+'", '
    item['exp_extract'] = [
                        # "[{timestamp}] {}txAtt:{txAtt:d}, {}torTemperature:{torTemperature:d} {}avgIMpa0:{avgIMpa0:d} {}",
                        # "{}[{timestamp}] {}Pma:{Pma:f}[{}DpdPma:{DpdPma:f}[{}Pmb:{Pmb:f}[{}TorPmb:{TorPmb:f}{}avgTxPma:{avgTxPma:f} {}",
                        "[{timestamp}] {}" 
                        ]
    # item['exp_mark'] = [
    #                         {"abbr":"ED","exp":"EVENT_DEACTIVATE","color":"#e00000"},
    #                         {"abbr":"ER","exp":"EVENT_RELEASE","color":"#d00000"},
    #                         {"abbr":"ES","exp":"EVENT_SETUP","color":"#c00000"},
    #                         {"abbr":"EA","exp":"EVENT_ACTIVATE","color":"#b00000"},
    #                         {"abbr":"IND","exp":"IND_ACTIVATE","color":"#a00000"}
    #                     ]
    item['is_active'] = False
    item['is_case_sensitive'] = True
    item['forward_rows'] = 0
    item['backward_rows'] = 0
    config['search'].append(item)

# tmp_search = []
# processes = range(2)
# for index in processes:
#     item = {}
#     branch_num = str(index)
#     item['role_path'] = f'{product_name}.' + 'branch' + branch_num + '.search_branch' + branch_num
#     item['identifier'] = 'search_branch' + branch_num
#     item['desc'] = 'Branch ' + branch_num + ' common use KeyValue and Mark'
#     item['exp_search'] = f'(process = "txlProcBranch'+branch_num+'"|process = "TxBranchCtrl'+branch_num+'")'
#     item['exp_extract'] = [
#                         "[{timestamp}] {}txAtt:{txAtt:d}, {}torTemperature:{torTemperature:d} {}avgIMpa0:{avgIMpa0:d} {}",
#                         # "{}[{timestamp}] {}Pma:{Pma:f}[{}DpdPma:{DpdPma:f}[{}Pmb:{Pmb:f}[{}TorPmb:{TorPmb:f}{}avgTxPma:{avgTxPma:f} {}",
#                         "[{timestamp}] {}" 
#                         ]
#     item['exp_mark'] = [
#                             {"abbr":"ED","exp":"EVENT_DEACTIVATE","color":"#759aa0"},
#                             {"abbr":"ER","exp":"EVENT_RELEASE","color":"#e69d87"},
#                             {"abbr":"ES","exp":"EVENT_SETUP","color":"#8dc1a9"},
#                             {"abbr":"EA","exp":"EVENT_ACTIVATE","color":"#ea7e53"},
#                             {"abbr":"IND","exp":"IND_ACTIVATE","color":"#eedd78"}
#                         ]
#     item['is_active'] = False
#     item['is_case_sensitive'] = True
#     item['forward_rows'] = 0
#     item['backward_rows'] = 0
#     config['search'].append(item)
#     tmp_search.append(item)
        
# keys = ['txAtt', 'avgIMpa0', 'ED', 'ER', 'ES', 'EA', 'IND']
# for search_config in tmp_search:
#     item = {}
#     item['role_path'] = search_config['role_path'].replace('search', 'chart')
#     item['identifier'] = search_config['identifier'].replace('search', 'chart')
#     item['desc'] = item['identifier'] + ' all KeyValue and Mark'
#     tree = {'namespace': 'KeyValue', 'name':'Key Value', 'check':False, 'children':[]}
#     for search_config2 in tmp_search:
#         search_atom = {}
#         search_atom['namespace'] = search_config2['identifier']
#         search_atom['name'] = search_config2['identifier']
#         search_atom['check'] = False
#         search_atom['children'] = []
#         for key in keys:
#             if search_config['identifier'] == search_config2['identifier']:
#                 search_atom['children'].append({'name': key, 'check': True})
#             else:
#                 search_atom['children'].append({'name': key, 'check': False})
#         tree['children'].append(search_atom)
#     item['key_value_tree'] = [{'id': '', 'type': 'TidyTree', 'elements':tree}]
#     config['chart'].append(item)
    
json_object = json.dumps(config)
with open("D:\\projects\\ericsson_flow\\new_files\\4449Config.ecfg", "w") as outfile:
    outfile.write(json_object)

In [ ]:
###################### User Define ###############################
# DCGM directory Location
dcgm_dir_path = 'D:/projects/ericsson_flow/new_files/Dot4455/'

extract_paths = [
    '*_logfiles.zip/teread.log',
    '*_e2.log.gz/*_dcg_e2.log',
    '*_modump.zip/*_dcg_k.log.gz/*_dcg_k.log'
]

# Save directory Location
save_path = 'D:/projects/ericsson_flow/new_dcgm_output/'
# Python regular express
telog_filters = 'radio6626'
# Python regular express
elog_filters = 'HW fault'
# Whether to integrate into one file.
is_into_one_file = False


class ZipDecompress(object):
    def __init__(self, path, extract_paths):
        self.path = path
        self.extract_paths = extract_paths
        self.result = {}
        self.decompress()
        
    def decompress(self):
        for num, package_name in enumerate(iterate_files_in_directory(self.path)):
            if '.zip' in package_name:
                self.result[package_name] = []
                zip_path = self.path + package_name
                with zipfile.ZipFile(zip_path, 'r') as file:
                    for extract_path in extract_paths:
                        file_name, lines = self.decompress_file(file, extract_path)
                        self.result[package_name].append([file_name, lines])
        
    def decompress_file(self, f_refer, extract_path):
        path_node = extract_path.split('/')[0].replace('*', '')
        tg = ''
        for file in f_refer.filelist:
            if path_node in file.filename:
                tg = file.filename

        if '.zip' in tg:
            with f_refer.open(tg, 'r') as nest:
                zfile = io.BytesIO(nest.read())
                with zipfile.ZipFile(zfile) as nested_zip:
                    return self.decompress_file(nested_zip, '/'.join(extract_path.split('/')[1:]))
        elif '.gz' in tg:
            with f_refer.open(tg, 'r') as nest:
                zfile = io.BytesIO(nest.read())
                lines = gzip.open(zfile, 'r')
                return nest.name, [line.decode("utf-8") for line in lines]
        else:
            with f_refer.open(tg, 'r') as nest:
                lines = nest.readlines()
                return tg, [line.decode("utf-8") for line in lines]


class ContentSearch(object):
    def __init__(self, lines):
        self.lines = lines
        self.result = {}
        
    def search_by_range(self, start_exp, end_exp):
        self.result = []
        
        flag = False
        lines = []
        indices = []
        start_exp_parse = ''
        end_exp_parse = ''
        for index, line in enumerate(self.lines):
            if flag:
                r = parse(end_exp, line)
                if r is not None:
                    flag = False
                    end_exp_parse = r.named
                    self.result.append({'start_exp_parse': start_exp_parse, 'end_exp_parse': end_exp_parse, 'indices': indices, 'lines': lines})
                    lines = []
                    indices = []
                else:
                    lines.append(line)
                    indices.append(index)
                    continue
            r = parse(start_exp, line)
            if r is not None:
                start_exp_parse = r.named
                flag = True
                lines.append(line)
                indices.append(index)
        return self.result
                    
    def search_by_regular(self):
        pass

In [ ]:
from utils import *

class KeyValueExtract(object):
    def __init__(self, lines):
        self.lines = lines
        self.result = {}
                        
    def extract_in_embedded(self, kv_exps):
        self.result = {}
        for index, line in enumerate(self.lines):
            for kv_exp in kv_exps:
                r = parse(kv_exp, line)
                if r is not None:
                    for key in r.named.keys():
                        if key not in self.result:
                            self.result[key] = []
                        self.result[key].append({'index': index, 'value': r.named[key], 'timestamp': convert_datetime_timestamp(r.named['timestamp'])})
        return self.result
    
    def extract_in_batch(self, kv_exps):
        timestamp = ''
        self.result = {}
        for index, line in enumerate(self.lines):
            for kv_exp in kv_exps:
                r = parse(kv_exp, line)
                if r is not None:
                    if 'timestamp' in r.named:
                        timestamp = convert_datetime_timestamp(r.named['timestamp'])
                        break
                    for key in r.named.keys():
                        if key not in self.result:
                            self.result[key] = []
                        self.result[key].append({'index': index, 'value': r.named[key], 'timestamp': timestamp})
        return self.result
    
    def extract_in_batch_notime(self, kv_exps):
        timestamp = ''
        self.result = {}
        for index, line in enumerate(self.lines):
            for kv_exp in kv_exps:
                r = parse(kv_exp, line)
                if r is not None:
                    for key in r.named.keys():
                        if key not in self.result:
                            self.result[key] = []
                        self.result[key].append({'index': index, 'value': r.named[key], 'timestamp': 0})
        return self.result

In [ ]:
###################### User Define ###############################
# DCGM directory Location
dir_path = 'D:/projects/ericsson_flow/new_files/AIR_HC/'
origin_data_output = 'D:/projects/ericsson_flow/new_files/RRU4432B28OriginData.csv'
product_name = 'RRU4432B28'
map_table_start_mark = ['FRU','LNH','BOARD','ST','FAULT','OPER','MAINT']
map_table_end_mark = '---------------------------------------------------------------------'
map_table_exp = '{} ;{assigned_name} ;{product_name} {}'

exps = {
    'trx status':{'kv_exps': [
                    '{}Date: {timestamp},{}',
                    '{}txPma{}: {txPma},{}',
                    '{}txDpdPma{}: {txDpdPma},{}',
                    '{}txPmb{}: {txPmb},{}',
                    '{}txTorPmb{}: {txTorPmb},{}',
                    '{}dpd {}: {dpd},{}',
                    '{}dpdStateMachine{}: {dpdStateMachine},{}', 
                    '{}gainStateMachine{}: {gainStateMachine},{}',
                    '{}linearizationStateMachine{}: {linearizationStateMachine},{}'
                ], 'filter_exps': {
                    'txPma': "row['txPma_abnormal'] = False",
                    'txDpdPma': "row['txDpdPma_abnormal'] = True if float(row['txPma']['value']) - float(row['txDpdPma']['value']) > 4 else False",
                    'txPmb': "row['txPmb_abnormal'] = True if float(row['txPma']['value']) - float(row['txPmb']['value']) != 0 else False",
                    'txTorPmb': "row['txTorPmb_abnormal'] = True if float(row['txPma']['value']) - float(row['txTorPmb']['value']) > 0.1 else False",
                    'dpd': "row['dpd_abnormal'] = True if row['dpd']['value'] in ['off'] else False",
                    'dpdStateMachine': "row['dpdStateMachine_abnormal'] = True if row['dpdStateMachine']['value'] in ['OFF'] else False",
                    'gainStateMachine': "row['gainStateMachine_abnormal'] = True if row['gainStateMachine']['value'] in ['CtrlGainStateIdle:started', 'ns:stopped'] else False", 
                    'linearizationStateMachine': "row['linearizationStateMachine_abnormal'] = True if row['linearizationStateMachine']['value'] in ['ns:stopped'] else False"
                }, 'search_type': 'range', 'extract_type': 'batch', 'start_exp': 'coli>/fruacc/lhsh {assigned_name} trx status{}', 'end_exp': 'coli>/fruacc/lhsh{}'},
    'elog read':{'kv_exps': [
                    '{}[{timestamp}]{}Lin. fault port{LinFault}',
                    '{}[{timestamp}]{}POWER LOST{PowerLost}',
                    '{}[{timestamp}]{}PSU enters{PsuEnter}',
                    '{}[{timestamp}]{}JESD LINK FAILURE{JesdLinkFailure}',
                    '{}[{timestamp}]{}#### DPDCONTROLLER{DpdController}',
                ], 'filter_exps': {
                    'LinFault': "",
                    'PowerLost': "",
                    'PsuEnter': "",
                    'JesdLinkFailure': "",
                    'DpdController': "",
                }, 'search_type': 'range', 'extract_type': 'embedded', 'start_exp': 'coli>/fruacc/lhsh {assigned_name} elog read{}', 'end_exp': '{}End of log{}'},
    'ts r':{'kv_exps': [
                    '{}Board{}: {Board} {}',
                    '{}DcPaVdd:0{}: {DcPaVdd_0} {}',
                    '{}DcPaVdd:1{}: {DcPaVdd_1} {}',
                    '{}DcTrxVcc{}: {DcTrxVcc} {}',
                    '{}Dra:0{}: {Dra_0} {}',
                    '{}PaDriver:A{}: {PaDriver_A} {}',
                    '{}PaDriver:B{}: {PaDriver_B} {}',
                    '{}PaDriver:C{}: {PaDriver_C} {}',
                    '{}PaDriver:D{}: {PaDriver_D} {}',
                    '{}PaFinal:A{}: {PaFinal_A} {}',
                    '{}PaFinal:B{}: {PaFinal_B} {}',
                    '{}PaFinal:C{}: {PaFinal_C} {}',
                    '{}PaFinal:D{}: {PaFinal_D} {}',
                    '{}Rx:A{}: {Rx_A} {}',
                    '{}Rx:B{}: {Rx_B} {}',
                    '{}Rx:C{}: {Rx_C} {}',
                    '{}Rx:D{}: {Rx_D} {}',
                    '{}Tor:A{}: {Tor_A} {}',
                    '{}Tor:B{}: {Tor_B} {}',
                    '{}Tor:C{}: {Tor_C} {}',
                    '{}Tor:D{}: {Tor_D} {}',
                    '{}Touch: {Touch} {}',
                    '{}TrxTemp:0{}: {TrxTemp_0} {}',
                    '{}Tx:A{}: {Tx_A} {}',
                    '{}Tx:B{}: {Tx_B} {}',
                    '{}Tx:C{}: {Tx_C} {}',
                    '{}Tx:D{}: {Tx_D} {}',
                ], 'filter_exps': {
                    'Board': "row['Board_abnormal'] = True if (float(row['Board']['value']) < 0) | (float(row['Board']['value']) > 50) else False",
                    'DcPaVdd_0': "row['DcPaVdd_0_abnormal'] = True if (float(row['DcPaVdd_0']['value']) < 0) | (float(row['DcPaVdd_0']['value']) > 60) else False",
                    'DcPaVdd_1': "row['DcPaVdd_1_abnormal'] = True if (float(row['DcPaVdd_1']['value']) < 0) | (float(row['DcPaVdd_1']['value']) > 60) else False",
                    'DcTrxVcc': "row['DcTrxVcc_abnormal'] = True if (float(row['DcTrxVcc']['value']) < 0) | (float(row['DcTrxVcc']['value']) > 40) else False",
                    'Dra_0': "row['Dra_0_abnormal'] = True if (float(row['Dra_0']['value']) < 0) | (float(row['Dra_0']['value']) > 60) else False",
                    'PaDriver_A': "row['PaDriver_A_abnormal'] = True if (float(row['PaDriver_A']['value']) < 0) | (float(row['PaDriver_A']['value']) > 70) else False",
                    'PaDriver_B': "row['PaDriver_B_abnormal'] = True if (float(row['PaDriver_B']['value']) < 0) | (float(row['PaDriver_B']['value']) > 70) else False",
                    'PaDriver_C': "row['PaDriver_C_abnormal'] = True if (float(row['PaDriver_C']['value']) < 0) | (float(row['PaDriver_C']['value']) > 70) else False",
                    'PaDriver_D': "row['PaDriver_D_abnormal'] = True if (float(row['PaDriver_D']['value']) < 0) | (float(row['PaDriver_D']['value']) > 70) else False",
                    'PaFinal_A': "row['PaFinal_A_abnormal'] = True if (float(row['PaFinal_A']['value']) < 0) | (float(row['PaFinal_A']['value']) > 80) else False",
                    'PaFinal_B': "row['PaFinal_B_abnormal'] = True if (float(row['PaFinal_B']['value']) < 0) | (float(row['PaFinal_B']['value']) > 80) else False",
                    'PaFinal_C': "row['PaFinal_C_abnormal'] = True if (float(row['PaFinal_C']['value']) < 0) | (float(row['PaFinal_C']['value']) > 80) else False",
                    'PaFinal_D': "row['PaFinal_D_abnormal'] = True if (float(row['PaFinal_D']['value']) < 0) | (float(row['PaFinal_D']['value']) > 80) else False",
                    'Rx_A': "row['Rx_A_abnormal'] = True if (float(row['Rx_A']['value']) < 0) | (float(row['Rx_A']['value']) > 55) else False",
                    'Rx_B': "row['Rx_B_abnormal'] = True if (float(row['Rx_B']['value']) < 0) | (float(row['Rx_B']['value']) > 55) else False",
                    'Rx_C': "row['Rx_C_abnormal'] = True if (float(row['Rx_C']['value']) < 0) | (float(row['Rx_C']['value']) > 55) else False",
                    'Rx_D': "row['Rx_D_abnormal'] = True if (float(row['Rx_D']['value']) < 0) | (float(row['Rx_D']['value']) > 55) else False",
                    'Tor_A': "row['Tor_A_abnormal'] = True if (float(row['Tor_A']['value']) < 0) | (float(row['Tor_A']['value']) > 55) else False",
                    'Tor_B': "row['Tor_B_abnormal'] = True if (float(row['Tor_B']['value']) < 0) | (float(row['Tor_B']['value']) > 55) else False",
                    'Tor_C': "row['Tor_C_abnormal'] = True if (float(row['Tor_C']['value']) < 0) | (float(row['Tor_C']['value']) > 55) else False",
                    'Tor_D': "row['Tor_D_abnormal'] = True if (float(row['Tor_D']['value']) < 0) | (float(row['Tor_D']['value']) > 55) else False",
                    'TrxTemp_0': "row['TrxTemp_0_abnormal'] = True if (float(row['TrxTemp_0']['value']) < 0) | (float(row['TrxTemp_0']['value']) > 75) else False",
                    'Tx_A': "row['Tx_A_abnormal'] = True if (float(row['Tx_A']['value']) < 0) | (float(row['Tx_A']['value']) > 55) else False",
                    'Tx_B': "row['Tx_B_abnormal'] = True if (float(row['Tx_B']['value']) < 0) | (float(row['Tx_B']['value']) > 55) else False",
                    'Tx_C': "row['Tx_C_abnormal'] = True if (float(row['Tx_C']['value']) < 0) | (float(row['Tx_C']['value']) > 55) else False",
                    'Tx_D': "row['Tx_D_abnormal'] = True if (float(row['Tx_D']['value']) < 0) | (float(row['Tx_D']['value']) > 55) else False",
                }, 'search_type': 'range', 'extract_type': 'batch_notime', 'start_exp': 'coli>/fruacc/lhsh {assigned_name} ts r{}', 'end_exp': '{}================{}'},
    'cs r':{'kv_exps': [
                    '{}PaDriver0:A{}: {PaDriver0_A} {}',
                    '{}PaDriver0:B{}: {PaDriver0_B} {}',
                    '{}PaDriver0:C{}: {PaDriver0_C} {}',
                    '{}PaDriver0:D{}: {PaDriver0_D} {}',
                    '{}PaDriver1:A{}: {PaDriver1_A} {}',
                    '{}PaDriver1:B{}: {PaDriver1_B} {}',
                    '{}PaDriver1:C{}: {PaDriver1_C} {}',
                    '{}PaDriver1:D{}: {PaDriver1_D} {}',
                    '{}PaFinal0:A{}: {PaFinal0_A} {}',
                    '{}PaFinal0:B{}: {PaFinal0_B} {}',
                    '{}PaFinal0:C{}: {PaFinal0_C} {}',
                    '{}PaFinal0:D{}: {PaFinal0_D} {}',
                    '{}PaFinal1:A{}: {PaFinal1_A} {}',
                    '{}PaFinal1:B{}: {PaFinal1_B} {}',
                    '{}PaFinal1:C{}: {PaFinal1_C} {}',
                    '{}PaFinal1:D{}: {PaFinal1_D} {}',
                ], 'filter_exps': {
                    'PaDriver0_A': "row['PaDriver0_A_abnormal'] = True if (float(row['PaDriver0_A']['value']) < 0.15) | (float(row['PaDriver0_A']['value']) > 0.25) else False",
                    'PaDriver0_B': "row['PaDriver0_B_abnormal'] = True if (float(row['PaDriver0_B']['value']) < 0.15) | (float(row['PaDriver0_B']['value']) > 0.25) else False",
                    'PaDriver0_C': "row['PaDriver0_C_abnormal'] = True if (float(row['PaDriver0_C']['value']) < 0.15) | (float(row['PaDriver0_C']['value']) > 0.25) else False",
                    'PaDriver0_D': "row['PaDriver0_D_abnormal'] = True if (float(row['PaDriver0_D']['value']) < 0.15) | (float(row['PaDriver0_D']['value']) > 0.25) else False",
                    'PaDriver1_A': "row['PaDriver1_A_abnormal'] = True if (float(row['PaDriver1_A']['value']) < 0.15) | (float(row['PaDriver1_A']['value']) > 0.25) else False",
                    'PaDriver1_B': "row['PaDriver1_B_abnormal'] = True if (float(row['PaDriver1_B']['value']) < 0.15) | (float(row['PaDriver1_B']['value']) > 0.25) else False",
                    'PaDriver1_C': "row['PaDriver1_C_abnormal'] = True if (float(row['PaDriver1_C']['value']) < 0.15) | (float(row['PaDriver1_C']['value']) > 0.25) else False",
                    'PaDriver1_D': "row['PaDriver1_D_abnormal'] = True if (float(row['PaDriver1_D']['value']) < 0.15) | (float(row['PaDriver1_D']['value']) > 0.25) else False",
                    'PaFinal0_A': "row['PaFinal0_A_abnormal'] = True if (float(row['PaFinal0_A']['value']) < 0.2) | (float(row['PaFinal0_A']['value']) > 0.7) else False",
                    'PaFinal0_B': "row['PaFinal0_B_abnormal'] = True if (float(row['PaFinal0_B']['value']) < 0.2) | (float(row['PaFinal0_B']['value']) > 0.7) else False",
                    'PaFinal0_C': "row['PaFinal0_C_abnormal'] = True if (float(row['PaFinal0_C']['value']) < 0.2) | (float(row['PaFinal0_C']['value']) > 0.7) else False",
                    'PaFinal0_D': "row['PaFinal0_D_abnormal'] = True if (float(row['PaFinal0_D']['value']) < 0.2) | (float(row['PaFinal0_D']['value']) > 0.7) else False",
                    'PaFinal1_A': "row['PaFinal1_A_abnormal'] = True if (float(row['PaFinal1_A']['value']) < 0.2) | (float(row['PaFinal1_A']['value']) > 0.7) else False",
                    'PaFinal1_B': "row['PaFinal1_B_abnormal'] = True if (float(row['PaFinal1_B']['value']) < 0.2) | (float(row['PaFinal1_B']['value']) > 0.7) else False",
                    'PaFinal1_C': "row['PaFinal1_C_abnormal'] = True if (float(row['PaFinal1_C']['value']) < 0.2) | (float(row['PaFinal1_C']['value']) > 0.7) else False",
                    'PaFinal1_D': "row['PaFinal1_D_abnormal'] = True if (float(row['PaFinal1_D']['value']) < 0.2) | (float(row['PaFinal1_D']['value']) > 0.7) else False",
                }, 'search_type': 'range', 'extract_type': 'batch_notime', 'start_exp': 'coli>/fruacc/lhsh {assigned_name} cs r{}', 'end_exp': '{}================{}'},
    'vs r':{'kv_exps': [
                    '{}Dc1_35{}: {Dc1_35} {}',
                    '{}Dc1_8{}: {Dc1_8} {}',
                    '{}Dc3_3{}: {Dc3_3} {}',
                    '{}Dc3_7{}: {Dc3_7} {}',
                    '{}Dc5_11{}: {Dc5_11} {}',
                    '{}DcAux{}: {DcAux} {}',
                    '{}DcTrxA1_3:0{}: {DcTrxA1_3_0} {}',
                    '{}DcTrxA1_8:0{}: {DcTrxA1_8_0} {}',
                    '{}DcTrxD1:0{}: {DcTrxD1_0} {}',
                    '{}DcTrxVcc{}: {DcTrxVcc} {}',
                    '{}DcXenon0_842:0{}: {DcXenon0_842_0} {}',
                    '{}DcXenon0_9:0{}: {DcXenon0_9_0} {}',
                    '{}DcXenonCore:0{}: {DcXenonCore_0} {}',
                    '{}PaDriverVdd:A{}: {PaDriverVdd_A} {}',
                    '{}PaDriverVdd:B{}: {PaDriverVdd_B} {}',
                    '{}PaDriverVdd:C{}: {PaDriverVdd_C} {}',
                    '{}PaDriverVdd:D{}: {PaDriverVdd_D} {}',
                    '{}PaFinalVdd:A{}: {PaFinalVdd_A} {}',
                    '{}PaFinalVdd:B{}: {PaFinalVdd_B} {}',
                    '{}PaFinalVdd:C{}: {PaFinalVdd_C} {}',
                    '{}PaFinalVdd:D{}: {PaFinalVdd_D} {}',
                    '{}Ret{}: {Ret} {}',
                ], 'filter_exps': {
                    'Dc1_35': "row['Dc1_35_abnormal'] = True if (float(row['Dc1_35']['value']) < 1.32) | (float(row['Dc1_35']['value']) > 1.38) else False",
                    'Dc1_8': "row['Dc1_8_abnormal'] = True if (float(row['Dc1_8']['value']) < 1.75) | (float(row['Dc1_8']['value']) > 1.85) else False",
                    'Dc3_3': "row['Dc3_3_abnormal'] = True if (float(row['Dc3_3']['value']) < 3.25) | (float(row['Dc3_3']['value']) > 3.35) else False",
                    'Dc3_7': "row['Dc3_7_abnormal'] = True if (float(row['Dc3_7']['value']) < 3.65) | (float(row['Dc3_7']['value']) > 3.75) else False",
                    'Dc5_11': "row['Dc5_11_abnormal'] = True if (float(row['Dc5_11']['value']) < 5.05) | (float(row['Dc5_11']['value']) > 5.15) else False",
                    'DcAux': "row['DcAux_abnormal'] = True if (float(row['DcAux']['value']) < 10.3) | (float(row['DcAux']['value']) > 10.5) else False",
                    'DcTrxA1_3_0': "row['DcTrxA1_3_0_abnormal'] = True if (float(row['DcTrxA1_3_0']['value']) < 1.27) | (float(row['DcTrxA1_3_0']['value']) > 1.33) else False",
                    'DcTrxA1_8_0': "row['DcTrxA1_8_0_abnormal'] = True if (float(row['DcTrxA1_8_0']['value']) < 1.77) | (float(row['DcTrxA1_8_0']['value']) > 1.83) else False",
                    'DcTrxD1_0': "row['DcTrxD1_0_abnormal'] = True if (float(row['DcTrxD1_0']['value']) < 0.99) | (float(row['DcTrxD1_0']['value']) > 1.05) else False",
                    'DcTrxVcc': "row['DcTrxVcc_abnormal'] = True if (float(row['DcTrxVcc']['value']) < 8.9) | (float(row['DcTrxVcc']['value']) > 9.1) else False",
                    'DcXenon0_842_0': "row['DcXenon0_842_0_abnormal'] = True if (float(row['DcXenon0_842_0']['value']) < 0.823) | (float(row['DcXenon0_842_0']['value']) > 0.883) else False",
                    'DcXenon0_9_0': "row['DcXenon0_9_0_abnormal'] = True if (float(row['DcXenon0_9_0']['value']) < 0.87) | (float(row['DcXenon0_9_0']['value']) > 0.93) else False",
                    'DcXenonCore_0': "row['DcXenonCore_0_abnormal'] = True if (float(row['DcXenonCore_0']['value']) < 0.711) | (float(row['DcXenonCore_0']['value']) > 0.755) else False",
                    'PaDriverVdd_A': "row['PaDriverVdd_A_abnormal'] = True if (float(row['PaDriverVdd_A']['value']) < 40) | (float(row['PaDriverVdd_A']['value']) > 51) else False",
                    'PaDriverVdd_B': "row['PaDriverVdd_B_abnormal'] = True if (float(row['PaDriverVdd_B']['value']) < 40) | (float(row['PaDriverVdd_B']['value']) > 51) else False",
                    'PaDriverVdd_C': "row['PaDriverVdd_C_abnormal'] = True if (float(row['PaDriverVdd_C']['value']) < 40) | (float(row['PaDriverVdd_C']['value']) > 51) else False",
                    'PaDriverVdd_D': "row['PaDriverVdd_D_abnormal'] = True if (float(row['PaDriverVdd_D']['value']) < 40) | (float(row['PaDriverVdd_D']['value']) > 51) else False",
                    'PaFinalVdd_A': "row['PaFinalVdd_A_abnormal'] = True if (float(row['PaFinalVdd_A']['value']) < 40) | (float(row['PaFinalVdd_A']['value']) > 51) else False",
                    'PaFinalVdd_B': "row['PaFinalVdd_B_abnormal'] = True if (float(row['PaFinalVdd_B']['value']) < 40) | (float(row['PaFinalVdd_B']['value']) > 51) else False",
                    'PaFinalVdd_C': "row['PaFinalVdd_C_abnormal'] = True if (float(row['PaFinalVdd_C']['value']) < 40) | (float(row['PaFinalVdd_C']['value']) > 51) else False",
                    'PaFinalVdd_D': "row['PaFinalVdd_D_abnormal'] = True if (float(row['PaFinalVdd_D']['value']) < 40) | (float(row['PaFinalVdd_D']['value']) > 51) else False",
                    'Ret': "row['Ret_abnormal'] = True if (float(row['Ret']['value']) < 29.37) | (float(row['Ret']['value']) > 29.43) else False",
                }, 'search_type': 'range', 'extract_type': 'batch_notime', 'start_exp': 'coli>/fruacc/lhsh {assigned_name} vs r{}', 'end_exp': '{}================{}'},
    'rvc r':{'kv_exps': [
                    '{}RxRfGain:A{}: {RxRfGain_A}\n',
                    '{}RxRfGain:B{}: {RxRfGain_B}\n',
                    '{}RxRfGain:C{}: {RxRfGain_C}\n',
                    '{}RxRfGain:D{}: {RxRfGain_D}\n',
                    '{}RxRfGain0:A{}: {RxRfGain0_A}\n',
                    '{}RxRfGain0:B{}: {RxRfGain0_B}\n',
                    '{}RxRfGain0:C{}: {RxRfGain0_C}\n',
                    '{}RxRfGain0:D{}: {RxRfGain0_D}\n',
                    '{}TorAttenuator:A{}: {TorAttenuator_A}\n',
                    '{}TorAttenuator:B{}: {TorAttenuator_B}\n',
                    '{}TorAttenuator:C{}: {TorAttenuator_C}\n',
                    '{}TorAttenuator:D{}: {TorAttenuator_D}\n',
                    '{}TxAttenuation:A{}: {TxAttenuation_A}\n',
                    '{}TxAttenuation:B{}: {TxAttenuation_B}\n',
                    '{}TxAttenuation:C{}: {TxAttenuation_C}\n',
                    '{}TxAttenuation:D{}: {TxAttenuation_D}\n',
                    '{}TxAttenuation0:A{}: {TxAttenuation0_A}\n',
                    '{}TxAttenuation0:B{}: {TxAttenuation0_B}\n',
                    '{}TxAttenuation0:C{}: {TxAttenuation0_C}\n',
                    '{}TxAttenuation0:D{}: {TxAttenuation0_D}\n',
                    '{}TxGain:A{}: {TxGain_A}\n',
                    '{}TxGain:B{}: {TxGain_B}\n',
                    '{}TxGain:C{}: {TxGain_C}\n',
                    '{}TxGain:D{}: {TxGain_D}\n',
                ], 'filter_exps': {
                    'RxRfGain_A': "row['RxRfGain_A_abnormal'] = True if (float(row['RxRfGain_A']['value']) < 7) | (float(row['RxRfGain_A']['value']) > 13) else False",
                    'RxRfGain_B': "row['RxRfGain_B_abnormal'] = True if (float(row['RxRfGain_B']['value']) < 7) | (float(row['RxRfGain_B']['value']) > 13) else False",
                    'RxRfGain_C': "row['RxRfGain_C_abnormal'] = True if (float(row['RxRfGain_C']['value']) < 7) | (float(row['RxRfGain_C']['value']) > 13) else False",
                    'RxRfGain_D': "row['RxRfGain_D_abnormal'] = True if (float(row['RxRfGain_D']['value']) < 7) | (float(row['RxRfGain_D']['value']) > 13) else False",
                    'RxRfGain0_A': "row['RxRfGain0_A_abnormal'] = True if (float(row['RxRfGain0_A']['value']) < 228) | (float(row['RxRfGain0_A']['value']) > 228) else False",
                    'RxRfGain0_B': "row['RxRfGain0_B_abnormal'] = True if (float(row['RxRfGain0_B']['value']) < 228) | (float(row['RxRfGain0_B']['value']) > 228) else False",
                    'RxRfGain0_C': "row['RxRfGain0_C_abnormal'] = True if (float(row['RxRfGain0_C']['value']) < 228) | (float(row['RxRfGain0_C']['value']) > 228) else False",
                    'RxRfGain0_D': "row['RxRfGain0_D_abnormal'] = True if (float(row['RxRfGain0_D']['value']) < 228) | (float(row['RxRfGain0_D']['value']) > 228) else False",
                    'TorAttenuator_A': "row['TorAttenuator_A_abnormal'] = True if (float(row['TorAttenuator_A']['value']) < 218) | (float(row['TorAttenuator_A']['value']) > 247) else False",
                    'TorAttenuator_B': "row['TorAttenuator_B_abnormal'] = True if (float(row['TorAttenuator_B']['value']) < 218) | (float(row['TorAttenuator_B']['value']) > 247) else False",
                    'TorAttenuator_C': "row['TorAttenuator_C_abnormal'] = True if (float(row['TorAttenuator_C']['value']) < 218) | (float(row['TorAttenuator_C']['value']) > 247) else False",
                    'TorAttenuator_D': "row['TorAttenuator_D_abnormal'] = True if (float(row['TorAttenuator_D']['value']) < 218) | (float(row['TorAttenuator_D']['value']) > 247) else False",
                    'TxAttenuation_A': "row['TxAttenuation_A_abnormal'] = True if (float(row['TxAttenuation_A']['value']) < 9) | (float(row['TxAttenuation_A']['value']) > 25) else False",
                    'TxAttenuation_B': "row['TxAttenuation_B_abnormal'] = True if (float(row['TxAttenuation_B']['value']) < 9) | (float(row['TxAttenuation_B']['value']) > 25) else False",
                    'TxAttenuation_C': "row['TxAttenuation_C_abnormal'] = True if (float(row['TxAttenuation_C']['value']) < 9) | (float(row['TxAttenuation_C']['value']) > 25) else False",
                    'TxAttenuation_D': "row['TxAttenuation_D_abnormal'] = True if (float(row['TxAttenuation_D']['value']) < 9) | (float(row['TxAttenuation_D']['value']) > 25) else False",
                    'TxAttenuation0_A': "row['TxAttenuation0_A_abnormal'] = True if (float(row['TxAttenuation0_A']['value']) < 1300) | (float(row['TxAttenuation0_A']['value']) > 1300) else False",
                    'TxAttenuation0_B': "row['TxAttenuation0_B_abnormal'] = True if (float(row['TxAttenuation0_B']['value']) < 1300) | (float(row['TxAttenuation0_B']['value']) > 1300) else False",
                    'TxAttenuation0_C': "row['TxAttenuation0_C_abnormal'] = True if (float(row['TxAttenuation0_C']['value']) < 1300) | (float(row['TxAttenuation0_C']['value']) > 1300) else False",
                    'TxAttenuation0_D': "row['TxAttenuation0_D_abnormal'] = True if (float(row['TxAttenuation0_D']['value']) < 1300) | (float(row['TxAttenuation0_D']['value']) > 1300) else False",
                    'TxGain_A': "row['TxGain_A_abnormal'] = True if (float(row['TxGain_A']['value']) < 1000) | (float(row['TxGain_A']['value']) > 3000) else False",
                    'TxGain_B': "row['TxGain_B_abnormal'] = True if (float(row['TxGain_B']['value']) < 1000) | (float(row['TxGain_B']['value']) > 3000) else False",
                    'TxGain_C': "row['TxGain_C_abnormal'] = True if (float(row['TxGain_C']['value']) < 1000) | (float(row['TxGain_C']['value']) > 3000) else False",
                    'TxGain_D': "row['TxGain_D_abnormal'] = True if (float(row['TxGain_D']['value']) < 1000) | (float(row['TxGain_D']['value']) > 3000) else False",
                }, 'search_type': 'range', 'extract_type': 'batch_notime', 'start_exp': 'coli>/fruacc/lhsh {assigned_name} rvc r{}', 'end_exp': '{}================{}'},

}

def abnormal_condition(row, code):
    exec(code)
    return row

origin_data = ''
if os.path.exists(origin_data_output):
    origin_data = pd.read_csv(origin_data_output)
    finished_files = list(set(origin_data.file_name.values))

for file_index, file_name in enumerate(iterate_files_in_directory(dir_path)):
    if type(origin_data) != str:
        if file_name in finished_files:
            continue
    
    path = f'{dir_path}/{file_name}'
    print(f'{file_index} Start handle {file_name}.')
    with open(path, 'r') as f:
        try:
            lines = f.readlines()
        except Exception as e:
            print(str(e))
            continue

        # find map table
        flag = False
        product_flag = False
        table = {}
        for line in lines:
            if list(set([True if word in line else False  for word in map_table_start_mark]))[0] == True:
                flag = True
            if flag:
                r = parse(map_table_exp, line)
                if r is not None:
                    table[r.named['assigned_name'].strip()] = r.named['product_name'].strip()
                    if product_name == r.named['product_name']:
                        product_flag = True
            if (map_table_end_mark in line) & flag:
                break
                
        if not product_flag:
            continue
            
        result = []
        cs = ContentSearch(lines)
        for cmd in exps.keys():
            search_contents = cs.search_by_range(exps[cmd]['start_exp'], exps[cmd]['end_exp'])
            cmd_batch = {}
            for search_content in search_contents:
                if search_content['start_exp_parse']['assigned_name'] not in table:
                    continue
                if product_name != table[search_content['start_exp_parse']['assigned_name']]:
                    continue

                if search_content['start_exp_parse']['assigned_name'] not in cmd_batch:
                    cmd_batch[search_content['start_exp_parse']['assigned_name']] = 0
                else:
                    cmd_batch[search_content['start_exp_parse']['assigned_name']] += 1
                kve = KeyValueExtract(search_content['lines'])
                if exps[cmd]['extract_type'] == 'batch':
                    kv = kve.extract_in_batch(exps[cmd]['kv_exps'])
                    for keyword in exps[cmd]['filter_exps'].keys():
                        an = keyword+'_abnormal'
                        ptmp = pd.DataFrame(kv)
                        if len(ptmp) == 0:
                            break
                        ptmp = ptmp.apply(lambda x: abnormal_condition(x, exps[cmd]['filter_exps'][keyword]), axis=1)
                        abnormal = []
                        for dot in list(ptmp.loc[(ptmp[an] == True), keyword].values):
                            dot['global_index'] = search_content['indices'][dot['index']]
                            dot['search_index'] = dot.pop('index')
                            abnormal.append(dot)
                        result.append({'file_name':file_name, 'product_name':product_name, 'assigned_name': search_content['start_exp_parse']['assigned_name'], 'cmd':cmd , 'cmd_batch':cmd_batch[search_content['start_exp_parse']['assigned_name']],
                                      'keyword': keyword, 'abnormal': abnormal})
                elif exps[cmd]['extract_type'] == 'batch_notime':
                    kv = kve.extract_in_batch_notime(exps[cmd]['kv_exps'])
                    for keyword in exps[cmd]['filter_exps'].keys():
                        an = keyword+'_abnormal'
                        ptmp = pd.DataFrame(kv)
                        if len(ptmp) == 0:
                            break
                        ptmp = ptmp.apply(lambda x: abnormal_condition(x, exps[cmd]['filter_exps'][keyword]), axis=1)
                        abnormal = []
                        for dot in list(ptmp.loc[(ptmp[an] == True), keyword].values):
                            dot['global_index'] = search_content['indices'][dot['index']]
                            dot['search_index'] = dot.pop('index')
                            abnormal.append(dot)
                        result.append({'file_name':file_name, 'product_name':product_name, 'assigned_name': search_content['start_exp_parse']['assigned_name'], 'cmd':cmd , 'cmd_batch':cmd_batch[search_content['start_exp_parse']['assigned_name']],
                                      'keyword': keyword, 'abnormal': abnormal})
                elif exps[cmd]['extract_type'] == 'embedded':
                    kv = kve.extract_in_embedded(exps[cmd]['kv_exps'])
                    for keyword in exps[cmd]['filter_exps'].keys():
                        abnormal = []
                        if keyword in kv:
                            for dot in kv[keyword]:
                                dot['global_index'] = search_content['indices'][dot['index']]
                                dot['search_index'] = dot.pop('index')
                                abnormal.append(dot)
                        result.append({'file_name':file_name, 'product_name':product_name, 'assigned_name': search_content['start_exp_parse']['assigned_name'], 'cmd':cmd , 'cmd_batch':cmd_batch[search_content['start_exp_parse']['assigned_name']],
                                      'keyword': keyword, 'abnormal': abnormal})
    pd.DataFrame(result).to_csv(origin_data_output, index=False, mode='a', header=not os.path.exists(origin_data_output))

In [ ]:
type('')

In [ ]:
table = pd.read_csv(origin_data_output)
table

In [ ]:
kve = KeyValueExtract(search_result['trx status'][0]['lines'])
kve.extract_in_batch(extract_exps['trx status']['kv_exps'])

In [ ]:
dcgm = 'D:/projects/ericsson_flow/new_files/Dot4455/AKH167_242367_230215_112831_CET_MSRBS-LN_CXP2010174-1_R56J19_dcgm.zip'
configs = {
    'telog':{'extract_path': '*_logfiles.zip/teread.log', 'cmds':{
                'telog read':{'extract_exps': [], 
                             'time_type': 'batch', 'start_exp': 'coli>/fruacc/lhsh {assigned_name} trx status{}', 'end_exp': 'coli>/fruacc/lhsh{}'},
                            }},
    # 'elog':{'extract_path': '*_e2.log.gz/*_dcg_e2.log', 'cmds':[
    #             {'extract_exps': [], 
    #              'cmd':'', 'time_type': 'batch', 'start_exp': 'coli>/fruacc/lhsh {assigned_name} trx status{}', 'end_exp': 'coli>/fruacc/lhsh{}'},
    #             ]},
    # 'modump':{'extract_path': '*_modump.zip/*_dcg_k.log.gz/*_dcg_k.log', 'cmds':[
    #             {'extract_exps': [], 
    #              'cmd':'', 'time_type': 'batch', 'start_exp': 'coli>/fruacc/lhsh {assigned_name} trx status{}', 'end_exp': 'coli>/fruacc/lhsh{}'},
    #             ]},
}

In [ ]:
#     # init 
#     files_cmds_content = {}
#     for file_name in configs.keys():
#         files_cmds_content[file_name] = {}
#         for cmd in configs[file_name]['cmds'].keys():
#             files_cmds_content[file_name][cmd] = {}

#     # find cmd content
#     for file_name in files_origin_lines.keys():
#         for cmd in configs[file_name]['cmds'].keys():
#             flag = False
#             lines = []
#             assigned_name = ''
#             for index, line in enumerate(files_origin_lines[file_name]):
#                 if flag:
#                     r = parse(cmd['end_exp'], line)
#                     if r is not None:
#                         flag = False
#                         files_cmds_content[file_name][cmd][assigned_name].append(lines)
#                         lines = []
#                     else:
#                         lines.append({'global_index':index, 'text':line})
#                         continue
#                 r = parse(cmd['start_exp'], line)
#                 if r is not None:
#                     if r['assigned_name'] not in files_cmds_content[file_name][cmd]:
#                         files_cmds_content[file_name][cmd][r['assigned_name']] = []
#                     flag = True
#                     assigned_name = r['assigned_name']
#                     lines.append({'global_index':index, 'text':line})

#     # extract key value
#     for file_name in files_origin_lines.keys():
#         for cmd in configs[file_name]['cmds'].keys():
#             for assigned in files_cmds_content[file_name][cmd].keys():
#                 cmd_batch = files_cmds_content[file_name][cmd][assigned]
#                 for batch_index, lines in enumerate(cmd_batch):
#                     tmp = {}
#                     cmd_config = configs[file_name]['cmds'][cmd]
#                     for extract_key in cmd_config['extract_exps'].keys():
#                         if extract_key != 'timestamp':
#                             tmp[extract_key] = []

#                     if cmd_config['time_type'] == 'embedded':
#                         for line in lines:
#                             for extract_key in cmd_config['extract_exps'].keys():
#                                 r = parse(cmd_config['extract_exps'][extract_key]['exp'], line['text'])
#                                 if r is not None:
#                                     item = {
#                                             'file_name':file_name, 'assigned_name':content['assigned_name'], 
#                                             'cmd':cmd, 'keywords':extract_key, 'global_index': line['global_index'], 'value':'',
#                                             'cmd_batch_index': batch_index, 'timestamp': convert_datetime_timestamp(r.named['timestamp']), 'x': random.randint(0, 100), 'y':random.randint(0, 100)
#                                             }
#                                     tmp[extract_key].append(item)
#                     elif exps[key]['time_type'] == 'batch':
#                         timestamp = ''
#                         for line in lines:
#                             for extract_key in cmd_config['extract_exps'].keys():
#                                 r = parse(cmd_config['extract_exps'][extract_key]['exp'], line['text'])
#                                 if r is not None:
#                                     if 'timestamp' in r.named:
#                                         timestamp = convert_datetime_timestamp(r.named['timestamp'])
#                                         break
#                                     item = {
#                                             'file_name':file_name, 'product_name':table[content['assigned_name']], 'assigned_name':content['assigned_name'], 
#                                             'cmd':key, 'keywords':extract_key, 'global_index': line['global_index'], 'value':r.named[extract_key],
#                                             'cmd_batch': cmd_batch, 'timestamp': timestamp, 'x': random.randint(0, 100), 'y':random.randint(0, 100)
#                                             }
#                                     tmp[extract_key].append(item)

#                     for keyword in tmp.keys():
#                         max_len = max([len(v) for v in tmp.values()])
#                         for k, v in tmp.items():
#                             if len(v) < max_len:
#                                 v.extend([None] * (max_len - len(v)))

#                         an = keyword+'_abnormal'
#                         ptmp = pd.DataFrame(tmp)
#                         if len(ptmp) == 0:
#                             break
#                         ptmp = ptmp.reset_index().rename(columns={'index':'occur_batch'})
#                         ptmp = ptmp.apply(lambda x: abnormal_condition(x, exps[key]['extract_exps'][keyword]['cond']), axis=1)
#                         abnormal = []
#                         for item,occur_batch  in ptmp.loc[(ptmp[an] == True), [keyword, 'occur_batch']].values:
#                             item['occur_batch'] = occur_batch
#                             abnormal.append(item)
#                         res.append({'file_name': file_name, 'product_name': table[content['assigned_name']],'assigned_name': content['assigned_name'], 'cmd': key, 'keywords': keyword, 'abnormal':abnormal})